In [24]:
import pandas as pd
df = pd.DataFrame({ 'gene':["1 // foo // blabla",
                                   "2 // bar // lalala",
                                   "3 // qux // trilil",
                                   "4 // woz // hohoho",
                                   "nopeynope"], 
                   'cnv':['a','b','c','d','e'],
                   'cell1':[5,9,1,7,9], 
                   'cell2':[12,90,13,87,9],
                   'location':['1', '13.4', '1.1-2', '1.2-p22', '43-q44'],
                  'np_t_w': ['1','13.4','1.1','2','3']})
df

,cell1,cell2,cnv,gene,location,np_t_w
0,5,12,a,1 // foo // blabla,1,1
1,9,90,b,2 // bar // lalala,13.4,13.4
2,1,13,c,3 // qux // trilil,1.1-2,1.1
3,7,87,d,4 // woz // hohoho,1.2-p22,2
4,9,9,e,nopeynope,43-q44,3


In [10]:
df['location'].str.extract('(-)', expand=True)

,0
0,NaN
1,NaN
2,-
3,-
4,-


In [91]:
df = pd.DataFrame([['a','b',1],
                  ['a','c',1.9],
                  ['a','d',45],
                  ['a','e',3.9],
                  ['b','c', 2.1],
                  ['b','d',-2],
                  ['b','e',4.1]],
                 columns = ['cnv','exp','np_t_w'])
df

,cnv,exp,np_t_w
0,a,b,1.0
1,a,c,1.9
2,a,d,45.0
3,a,e,3.9
4,b,c,2.1
5,b,d,-2.0
6,b,e,4.1


In [92]:
### OLD
from scipy.stats import pearsonr
def rolling_pearson_0(df, group='cnv', data='np_t_w'):
    """
    computes pearson between array of data for one gene partners and each successive chromosomal neighbor
    
    :param df: pd df, clove results, sorted in ascending order of chromosome locus
    :param group: str, df column label on which to form gene neigbors, default 'cnv'
    :param data: str, df column label of source data to populate arrays, default 'np_t_w' (clove t-stats)
    """
    
    unique_genes = df[group].unique()
    results = []
    for idx, g0 in enumerate(unique_genes):
        g0_arr = df[df[group] == g0][data].values
        if idx < len(unique_genes) - 1:
            for g1 in unique_genes[idx+1:]:
                g1_arr = df[df[group] == g1][data].values
                print(g1_arr)
                pearson, pval = pearsonr(g0_arr, g1_arr)
                results.append([g0, g1, pearson, pval])

    return pd.DataFrame(results, columns=[group, group+'_k', 'pearson', 'pval'])

In [95]:
### NEW
from scipy.stats import pearsonr
from scipy.spatial import distance
def rolling_similarity(df, group='cnv', data='np_t_w', how='pearson'):
    """
    computes similarity between array of data for one gene partners and each successive chromosomal neighbor
    
    :param df: pd df, clove results, sorted in ascending order of chromosome locus
    :param group: str, df column label on which to form gene neigbors, default 'cnv'
    :param data: str, df column label of source data to populate arrays, default 'np_t_w' (clove t-stats)
    :param how: str, choice of: {pearson, euclidian, cosine}
    """
    
    unique_genes = df[group].unique()
    results = []
    for idx, g0 in enumerate(unique_genes):
        if idx < len(unique_genes) - 1:
            g1 = unique_genes[idx+1]
            merged = pd.merge(df[df[group] == g0], df[df[group] == g1], how='inner', on='exp')
            merged.columns = ['cnv_g0','exp','clove_g0','cnv_g1','clove_g1']
            if how == 'pearson':
                cols = [g0, g1, how ,'pval']
                pear, pval = pearsonr(merged['clove_g0'], merged['clove_g1'])
                results.append([g0,g1,pear,pval])
            elif how == 'euclidian':
                cols = [g0, g1, how]
                results.append([g0, g1, distance.euclidean(merged['clove_g0'], merged['clove_g1'])])
            elif how == 'cosine':
                cols = [g0, g1, how]
                results.append([g0, g1, distance.cosine(merged['clove_g0'], merged['clove_g1'])])

    return pd.DataFrame(results, columns=cols)

In [94]:
rolling_similarity(df, group='cnv',data='np_t_w')

q


,a,b,pearson,pval
0,a,b,-0.932864,0.234603


In [27]:
df

,cell1,cell2,cnv,gene,location,np_t_w
0,5,12,a,1 // foo // blabla,1,1
1,9,90,b,2 // bar // lalala,13.4,13.4
2,1,13,c,3 // qux // trilil,1.1-2,1.1
3,7,87,d,4 // woz // hohoho,1.2-p22,2
4,9,9,e,nopeynope,43-q44,3


In [13]:
pearsonr?

In [22]:
import numpy as np
x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
x[:2]

array([0, 1])

In [23]:
np.random.shuffle(x)


TypeError: 'NoneType' object is not subscriptable